## Developing the artificial benchmark for federated learning spam detection

This aims at developing a benchmark for spam email detection using federated learning. The files will be generated\
with their name embedding one of the 2 classification categories: spam and not spam. This benchmark is parameterized to\
be able to quantify the difficulty.

### What is the alphabet used?
english alphabet and decimal digits abcdefghijklmnopkrstuvwxyz0123456789\
A total of $N = 2^{36}$ possible substrings made from our 36 alphanumeric characters

### How are words composed?
TODO: figure out a rule structure of the words

### How are sentences composed?
Sentences will be composed of $m$ words following a defined grammar. For simplicity, we start with \
$m = 3$ words arrange in $noun + verb + noun$ configuration.The configuration is defined by a gammar G.

### What is the grammar?
Concatenation of the words vectors

### How are emails composed?
Emails are considered paragraphs made of sentences. An email will be made of $k$ sentences.

### How is a spam defined?


## Evaluation on Real Data